In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-3/transformers/8b-chat-hf/1/model.safetensors.index.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model-00003-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-chat-hf/1/config.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/LICENSE
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model-00001-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/USE_POLICY.md
/kaggle/input/llama-3/transformers/8b-chat-hf/1/tokenizer.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/tokenizer_config.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/example_text_completion.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/test_tokenizer.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/requirements.txt
/kaggle/input/llama-3/transformers/8b-chat-hf/1/tokenizer.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model-00004-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-chat-hf

In [2]:
%pip install --upgrade --q  langchain langchain_experimental langchain-openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.6.1 requires cubinlinker, which is not installed.
cudf 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.6.1 requires ptxcompiler, which is not installed.
cuml 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.9.0 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is incompatible.
cudf 24.6.1 requires cuda-python<12.0a0,>=11.7.1, but you have cu

In [3]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_experimental.tabular_synthetic_data.openai import (
    OPENAI_TEMPLATE,
    create_openai_data_generator,
)
from langchain_experimental.tabular_synthetic_data.prompts import (
    SYNTHETIC_FEW_SHOT_PREFIX,
    SYNTHETIC_FEW_SHOT_SUFFIX,
)
from langchain_openai import ChatOpenAI
import pandas as pd


class VaccinationData(BaseModel):
    message: str
    response: str
    
        
from kaggle_secrets import UserSecretsClient


user_secrets = UserSecretsClient()
openai_key = user_secrets.get_secret("openai_key")


examples = [
    {
        "example": """Message: When should my child receive their first vaccination?
                      Response: Your child should receive their first vaccination, the Hepatitis B vaccine, at birth, preferably within 24 hours."""
    },
    {
        "example": """Message: Are vaccines safe for pregnant women?
                      Response: Yes, certain vaccines, like the influenza (flu) and Tdap vaccines, are safe and recommended during pregnancy to protect both the mother and the baby."""
    },
    {
        "example": """Message: What are the common side effects of vaccines?
                      Response: Common side effects of vaccines include mild fever, soreness at the injection site, and fatigue. These side effects typically resolve on their own within a few days."""
    },
]



OPENAI_TEMPLATE = PromptTemplate(input_variables=["example"], template="{example}")

prompt_template = FewShotPromptTemplate(
    prefix=SYNTHETIC_FEW_SHOT_PREFIX,
    examples=examples,
    suffix=SYNTHETIC_FEW_SHOT_SUFFIX,
    input_variables=["subject", "extra"],
    example_prompt=OPENAI_TEMPLATE,
)


synthetic_data_generator = create_openai_data_generator(
    output_schema=VaccinationData,
    llm=ChatOpenAI(
        temperature=0.4,
        api_key = openai_key
    ),  
    prompt=prompt_template,
)


synthetic_results = synthetic_data_generator.generate(
    subject="vaccination_data",
    extra="Generate vaccination questions and answers using accurate information from CDC, WHO, NPI Schedule, UNICEF, and PAN guidelines, ensuring clarity and reliability in the format provided.",
    runs=1000,
)

In [4]:
import pandas as pd

# Convert data to DataFrame
dfs = pd.DataFrame([(d.message, d.response) for d in synthetic_results], columns=['message', 'response'])

dfs.to_csv('vax_syn.csv', index=False)

dfs.head()

,message,response
0,When should children receive their first vacci...,Children should receive their first vaccinatio...
1,Are vaccines safe for pregnant women?,"Yes, certain vaccines, like the influenza (flu..."
2,What are the common side effects of vaccines?,Common side effects of vaccines include mild f...
3,When should children receive their first vacci...,Children should receive their first vaccinatio...
4,Are vaccines safe for pregnant women?,"Yes, certain vaccines, like the influenza (flu..."


In [5]:
df = pd.read_csv('/kaggle/input/vax-llama/vax-llama-data.csv')

In [6]:
df.sample(10)

,messager_number,message,response
80,2347061728434,So how will I refer??,Send the link to other nursing mothers so they...
1151,2349060534899,Good day to you sir/ma,🧑 Good day ma
737,2348103966621,"And we've gone for all our vaccines, thank you",Okay noted ma
748,2348068922000,I was told to drop my account details for assi...,Thank you for sharing this information.\nA mem...
908,2347069624888,I'm not seeing anything,We urge you to take your child for his or her ...
696,2347035139127,At oloshun Area,Where exactly ma?
912,2347042953738,"The center is too far from our house,so pls I ...",Please use our location based service to find ...
1038,2348111063319,1 and half years old,🧑 He/She still needs to collect immunization t...
225,2348135492457,The last he took was 1yr3month,His next vaccination should be at 18months ma
188,2348029927676,Good morning,Hello and welcome!\n\nYou've reached the *Help...


In [7]:
dfx= df.copy()

df.shape

(1373, 3)

In [8]:
# Dropping the 'messager_number' column
df.drop('messager_number', axis=1, inplace=True)

# Dropping duplicate rows based on the 'response' column
df.drop_duplicates(subset='response', keep='first', inplace=True)


In [9]:
df.sample(10)

,message,response
1194,I cannot be going to Ado Ekiti every time due ...,🧑 You can use our Location service to get the ...
1077,Please my baby will complete 15 month on 21 of...,"🧑 No ma, you must take on her next appointed d..."
13,How do I track my child's vaccination date?,You can make use of our HelpMum Vaccination Tr...
79,August 30th,This is the right one ma
20,Stand a chance to win an amazing prize,"Stand a chance to win 200,000 when you downloa..."
1238,Where are you chatting from,"🧑 Our office is in Abobo, Ibadan."
334,Hope it is not late ma/ sir,it is not late ma
465,please hope he can still take it by December,How old is he ma?
229,Thank you for your message. We're unavailable ...,Hope the immunization date is correct ma
1219,I need 50k please to start my business,🧑 We do not render that kind of service ma. We...


In [10]:
df.shape

(619, 2)

In [11]:
df2 = pd.read_csv('/kaggle/input/vaxai-cloud/vaxai-cloud-data.csv')


df2.sample(10)

,1,hello,Hello! How can I assist you with vaccination and immunization today?,1.1,17-15ifQuSy24t1PhwK,2024-04-07 22:08:00.170841+00
259,258,what is a vaccine,A vaccine is a substance that stimulates a per...,3,3-1,2024-05-07 21:03:52.910243+00
42,44,Tell me more about parasites,I'm specialized in vaccination and immunizatio...,3,39-Op5xFNLNO0QpnUB8,2024-04-09 22:36:31.447043+00
167,168,What is a vaccine,A vaccine is a substance that stimulates the b...,3,3-7399,2024-05-01 10:08:22.663242+00
359,361,What can be done for me,Kindly send your details so we can understand ...,3,3-4245,2024-05-17 08:07:59.958392+00
126,127,but then how do their immune systems respond w...,When a pregnant woman receives the DTaP vaccin...,3,56-alXfJQC1ST1BDzqm,2024-04-25 09:01:36.029754+00
591,592,why is that,Could you please specify what you're asking ab...,3,3-1,2024-06-27 06:28:32.832848+00
261,260,what is a vaccine,A vaccine is a substance that stimulates a per...,3,3-1,2024-05-07 21:06:17.717103+00
100,101,Did they use open AI data,I can't confirm the specifics of the datasets ...,3,36-DEg9wKuTGS5M7Y3y,2024-04-18 22:08:25.874323+00
103,104,I thought you were trained on OpenAI,I was designed by the HelpMum AI Team and not ...,3,36-GpA69V8ohQiZ1fhM,2024-04-19 17:14:49.236124+00
431,433,What type of side effects can my child get due...,Common side effects from the Vitamin A supplem...,3,3-3954,2024-06-12 13:16:58.979849+00


In [12]:
# Append the current column names as a new row at the top
new_row = pd.DataFrame([df2.columns.tolist()], columns=df2.columns)
df2 = pd.concat([new_row, df2], ignore_index=True)

df2.head(10)

,1,hello,Hello! How can I assist you with vaccination and immunization today?,1.1,17-15ifQuSy24t1PhwK,2024-04-07 22:08:00.170841+00
0,1,hello,Hello! How can I assist you with vaccination a...,1.1,17-15ifQuSy24t1PhwK,2024-04-07 22:08:00.170841+00
1,2,how can I get malaria vaccine,You can get the malaria vaccine at any primary...,1,17-15ifQuSy24t1PhwK,2024-04-07 22:12:23.918852+00
2,3,okay thanks,You're welcome! If you have any more questions...,1,17-15ifQuSy24t1PhwK,2024-04-07 22:12:37.654053+00
3,4,I want to know more about typhoid vaccine,The typhoid vaccine helps protect against typh...,1,17-15ifQuSy24t1PhwK,2024-04-07 22:12:57.993688+00
4,5,hello,Hello! How can I assist you with vaccination a...,3,1-EucaCmzD6mwKgVoM,2024-04-08 06:03:17.087349+00
5,6,Hello],Hello! How can I assist you with vaccination o...,3,36-rPz1smepldvS1d6w,2024-04-08 19:56:43.554295+00
6,7,How are you doing,I'm here to help you with your questions about...,3,36-rPz1smepldvS1d6w,2024-04-08 19:56:55.568892+00
7,8,I need an information on immunzation,"Sure, what specific information about immuniza...",3,36-rPz1smepldvS1d6w,2024-04-08 19:58:21.089467+00
8,9,What do you need to do about HelpMum Vaccination,"To get assistance with HelpMum Vaccination, ki...",3,36-rPz1smepldvS1d6w,2024-04-08 19:59:36.059031+00
9,10,My child started having a fever after collecti...,Mild side effects like fever are common after ...,3,36-kP1gw2DQOjKpbxbN,2024-04-08 20:00:59.302054+00


In [13]:
# Drop the first and the last two columns
df2 = df2.iloc[:, 1:-3]

# Rename the remaining columns
df2.columns = ['message', 'response']

# Dropping duplicate rows based on the 'message' column
df2.drop_duplicates(subset='message', keep='first', inplace=True)


In [14]:
df2.head(10)

,message,response
0,hello,Hello! How can I assist you with vaccination a...
1,how can I get malaria vaccine,You can get the malaria vaccine at any primary...
2,okay thanks,You're welcome! If you have any more questions...
3,I want to know more about typhoid vaccine,The typhoid vaccine helps protect against typh...
5,Hello],Hello! How can I assist you with vaccination o...
6,How are you doing,I'm here to help you with your questions about...
7,I need an information on immunzation,"Sure, what specific information about immuniza..."
8,What do you need to do about HelpMum Vaccination,"To get assistance with HelpMum Vaccination, ki..."
9,My child started having a fever after collecti...,Mild side effects like fever are common after ...
10,Any urgent thing i can do to resolve it,"For a fever after vaccination, you can give yo..."


In [15]:
df2.shape

(432, 2)

In [16]:
dfs.shape

(1000, 2)

In [17]:
# Join the 2 dataframes
df_merged = pd.concat([dfs, df2], ignore_index=True)
df_merged.sample(10)

,message,response
1264,tell me more about hpv vaccination,The HPV vaccine protects against the human pap...
1084,Are you sure about this,"Yes, I'm sure about my training sources."
553,Synthetic data about vaccination_data has been...,message: What is the recommended age for the f...
965,Synthetic data about vaccination_data has been...,message: What are the common side effects of t...
236,What are the common side effects of vaccines?,Common side effects of vaccines may include re...
913,Synthetic data about vaccination_data has been...,message: What is the recommended age for the f...
554,Synthetic data about vaccination_data has been...,message: What are the common side effects of t...
254,What are the common side effects of vaccines?,Common side effects of vaccines may include re...
977,Synthetic data about vaccination_data has been...,message: What are the common side effects of t...
62,What are the common side effects of vaccines?,Common side effects of vaccines may include re...


In [18]:
df_merged.shape

(1432, 2)

In [19]:
# Save df_merged to a CSV file
df_merged.to_csv('final1.csv', index=False)

In [20]:
# Install necessary libraries
!pip install -q transformers datasets huggingface_hub
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl
!pip install -q git+https://github.com/huggingface/accelerate.git
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q bitsandbytes
!pip install -q peft

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.


In [21]:
# Import libraries
from datasets import Dataset, load_dataset
import pandas as pd
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,Trainer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,AutoConfig,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os
import time
import torch
from datasets import Dataset
from huggingface_hub import notebook_login, HfFolder
from trl import SFTTrainer,setup_chat_format, SFTConfig
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training

2024-07-22 05:27:00.483574: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-22 05:27:00.483677: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 05:27:00.676928: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [22]:
from huggingface_hub import HfApi
from kaggle_secrets import UserSecretsClient


user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("hf_token")  


# Set the Hugging Face API token as an environment variable
os.environ["HF_HOME"] = "/root/.cache/huggingface"
os.environ["TRANSFORMERS_CACHE"] = "/root/.cache/huggingface/transformers"
os.environ["HF_DATASETS_CACHE"] = "/root/.cache/huggingface/datasets"
os.environ["HF_METRICS_CACHE"] = "/root/.cache/huggingface/metrics"
os.environ["HF_HUB_TOKEN"] = hf_token

os.environ["HF_HUB_TOKEN"] = hf_token

In [23]:
wandb_key = user_secrets.get_secret("wandb_api")
import wandb
! wandb login $wandb_key

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [42]:
# Define the location and configuration for the model
model_id = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

new_model = 'vax-llama-1'

In [43]:
compute_dtype = torch.bfloat16

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True
)

# Measure time for loading model and tokenizer
time_start = time.time()

# Load the model and tokenizer configurations
model_config = AutoConfig.from_pretrained(
    model_id,
    trust_remote_code=True,
    max_new_tokens=1024
)

# Load the model with quantization settings
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    attn_implementation='eager'
)


# Measure and display time taken to load
time_end = time.time()
print(f"Prepare model, tokenizer: {round(time_end - time_start, 3)} sec.")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Prepare model, tokenizer: 16.587 sec.


In [44]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Set up the chat format
model, tokenizer = setup_chat_format(model, tokenizer)
#model = prepare_model_for_kbit_training(model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [45]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

In [46]:
# Load the dataset
dataset = Dataset.from_pandas(df_merged, split="all")

def format_chat_template(row):
    row_json = [{"role": "user", "content": row["message"]},
               {"role": "assistant", "content": row["response"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc=4,
)

dataset['text'][3]

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/1432 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


'<|im_start|>user\nWhen should children receive their first vaccination?<|im_end|>\n<|im_start|>assistant\nChildren should receive their first vaccination, the Hepatitis B vaccine, at birth, preferably within 24 hours.<|im_end|>\n'

In [47]:
dataset['text'][5]

'<|im_start|>user\nWhat are the common side effects of vaccines?<|im_end|>\n<|im_start|>assistant\nCommon side effects of vaccines include mild fever, soreness at the injection site, and fatigue. These side effects typically resolve on their own within a few days.<|im_end|>\n'

In [48]:
dataset = dataset.train_test_split(test_size=0.1)

In [49]:
os.environ["WANDB_DISABLED"] = "false"

In [50]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

# Create the Trainer without deprecated arguments
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will over

Map:   0%|          | 0/1288 [00:00<?, ? examples/s]

Map:   0%|          | 0/144 [00:00<?, ? examples/s]

In [51]:
# Train the model
trainer.train()

Step,Training Loss,Validation Loss
387,1.765100,0.435181
774,0.034100,0.422152
1161,0.011200,0.374993
1548,0.012500,0.370552


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /kaggle/input/llama-3/transformers/8b-chat-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /kaggle/input/llama-3/transformers/8b-chat-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /kaggle/input/llama-3/transformers/8b-chat-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /kaggle/input/llama-3/transformers/8b-chat-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=1932, training_loss=0.3216029993831093, metrics={'train_runtime': 12436.3193, 'train_samples_per_second': 0.311, 'train_steps_per_second': 0.155, 'total_flos': 2.8384147431235584e+16, 'train_loss': 0.3216029993831093, 'epoch': 3.0})

In [52]:
# Evaluate the model
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.3554185628890991, 'eval_runtime': 195.8647, 'eval_samples_per_second': 0.735, 'eval_steps_per_second': 0.735, 'epoch': 3.0}


In [53]:
wandb.finish()
model.config.use_cache = True

eval/loss,██▁▁▁▁▁
eval/runtime,▁▁█████
eval/samples_per_second,██▁▁▁▁▁
eval/steps_per_second,██▁▁▁▁▁
train/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇██
train/global_step,▁▁▁▂▂▂▂▃▃▃▃▄▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇██
train/grad_norm,▄▂▁▂▁▁▁▁▁▁▁▁▁▁▆▁▁▁▃▇█▁▁▅▆▇▁▂▁▁▇▁▁▁▃▁▁▁▃▄
train/learning_rate,████▇▇▇▆▆▆▆▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▅▅▅▇▆▅▆▆▆▅▅▅▁▁▁▁▁▁▂▂▁▁▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁
eval/loss,0.35542
eval/runtime,195.8647


In [54]:
trainer.model.save_pretrained(new_model)

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /kaggle/input/llama-3/transformers/8b-chat-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(


In [55]:
messages = [
    {
        "role": "user",
        "content": "Are vaccines safe for pregnant women?"
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, 
                   truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=150, 
                         num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Yes, vaccines are safe for pregnant women and are recommended to protect both the mother and the baby from serious diseases. However, it is important for pregnant women to consult with their healthcare provider before getting vaccinated to ensure the safety and effectiveness of the vaccines during pregnancy. Some vaccines, like the influenza (flu) and Tdap vaccines, are safe and recommended during pregnancy to protect both the mother and the baby. Others, like the HPV vaccine, are not recommended during pregnancy due to the risk of serious side effects. It is important for pregnant women to discuss their vaccination schedule with their healthcare provider to ensure they are protected against serious diseases. Are vaccines safe for pregnant women? Yes,


In [56]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("hf_token_2") 


from huggingface_hub import HfApi, HfFolder

# Set the token
HfFolder.save_token(hf_token)  

api = HfApi()
whoami = api.whoami(token=hf_token)
print(f"Logged in as: {whoami['name']}")

Logged in as: HelpMum-Personal


In [57]:
trainer.model.push_to_hub(new_model, use_temp_dir=True, token=hf_token)

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/HelpMum-Personal/vax-llama-1/commit/033a45646e43221c0ce80e74a7b60ce3aa0efdef', commit_message='Upload model', commit_description='', oid='033a45646e43221c0ce80e74a7b60ce3aa0efdef', pr_url=None, pr_revision=None, pr_num=None)